In [ ]:
import yaml
import astropy.units as u
import numpy as np

import matplotlib.pyplot as plt

# Uncomment if SimBMVtool not in directory
# import sys
# path_SimBMVtool = '<path_to_SimBMVtool>'
# sys.path.append(path_SimBMVtool)

from SimBMVtool.simulator_creator import SimulatorCreator
from SimBMVtool.toolbox import (
    get_data_store,
)

path_config = './config_simu.yaml'

In [ ]:
# Uncomment if you haven't downloaded gammapy datasets and update the path in config_simu.yaml
# It is needed for the catalogs and example dataset

# !gammapy download datasets
# !conda env config vars set GAMMAPY_DATA=$PWD/gammapy-datasets/2.0

# You may need to restart the environment

In [ ]:
# For the simulation, SimBMVtool needs FULL-ENCLOSURE dl3 files with at least the exposure, edisp and aeff IRFs.

# In the config file, set the path_data and the run numbers of the used observations.
# If you don't already know which one you can use, check the obs_ids in the data store

# Check the files in the directory following config['paths']['path_data']
with open(path_config, 'r') as file:
    config = yaml.safe_load(file)
path_data = config["paths"]["path_data"]
obs_pattern = config["data"]["obs_pattern"]
source_name = config["source"]["catalog_name"]

data_store = get_data_store(path_data,obs_pattern)
dfobs_table = data_store.obs_table.to_pandas().set_index("OBS_ID")

# Look for the source in case there are several in the table.
# The objects are compared with lower case and no space to account for different names
print(f"Source: {source_name}")
is_source = dfobs_table.OBJECT.str.lower().str.replace(' ', '') == source_name.lower().replace(' ','')
display(dfobs_table.loc[is_source])

In [ ]:
simulator = SimulatorCreator(path_config)
simulator.load_true_background_irfs() 

# If you want to get the gammapy BackgroundIRF without knowing the name of the variables
true = simulator.get_background_irf(type='true',downsampled=False) 
true.plot_at_energy(["1 TeV"],figsize=(3,3))
plt.show()

# You can also plot directly from the class object.
if simulator.true_collection: simulator.bkg_true_irf_collection[0].peek()
else: simulator.bkg_true_irf.peek()

In [ ]:
# Simulation takes ~7 min 40s with example parameters.

do_simulation = True
if do_simulation: simulator.do_simulation() # Simulating gives a lot of warning, this is expected
else: simulator.load_observation_collection()

In [ ]:
if simulator.obs_collection[0].psf is not None: simulator.obs_collection[0].peek()
simulator.obs_collection[0].events.peek()

In [ ]:
# In the config file, Crab is the default simulated source, with a flux set to 0 for background only simulation
# Now let's add the source to simulation (it will take longer). 
# The parameters are changed from here but should be changed directly in the config file
# The config file is copied in the simulation directory, but it is not updated if you change the parameters from here
# TO-DO: method to export parameters to config file 
simulator.flux_to_0 = False
simulator.subdir = "crab_hess_with_source"
simulator.init_save_paths()

simulator.do_simulation()

In [ ]:
# Check the result
simulator.obs_collection[1].events.peek()

In [ ]:
# Now let's simulate a custom source. 
# The spectral and spatial models needs to be given as a gammapy SkyModel dictionary format.
# In the example config file, I took the Crab models and just changed the spatial model type from a point to a disk

# Here is how to do the simulation, but again this can be declared directly in the config file
from gammapy.modeling.models import SkyModel

simulator.flux_to_0 = False
simulator.is_custom = True
custom_source_dict = simulator.cfg_source["custom_source"]
custom_source_dict["name"] = "crab disk"
simulator.source_model = SkyModel.from_dict(custom_source_dict)

simulator.subdir = "crab_hess_with_disk_source"
simulator.init_save_paths()

# To save time and memory for the sake of this tutorial, let's simulate only 1 run with 1 wobble 
simulator.n_run = 1
simulator.single_pointing = True

simulator.do_simulation()

In [ ]:
print(simulator.source_model)
simulator.obs_collection[0].events.select_offset([0.*u.deg,2.*u.deg]).plot_image()